In [2]:
import os,json,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("aside")) # to access aside.experiments 
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import re
from tqdm import tqdm
from utils.utils import *
from datasets import load_dataset
import ast
from collections import defaultdict
from utils.model_utils import load_model
from vllm import SamplingParams
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # to suppress warning message

In [ ]:
# model_path = "Qwen/Qwen3-8B"
model_path = '/mnt/disk1/xulin/models/Qwen3-8B'
# tokenizer = AutoTokenizer.from_pretrained(model_path)
use_vllm = True
model,tokenizer,is_aside = load_model(model_path,use_vllm=use_vllm,dtype=torch.bfloat16,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':True}) # load model if generating the toucan dpo

INFO 10-09 11:51:44 [utils.py:328] non-default args: {'max_model_len': 32768, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enable_chunked_prefill': True, 'model': '/mnt/disk1/xulin/models/Qwen3-8B'}
INFO 10-09 11:51:44 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM
INFO 10-09 11:51:44 [__init__.py:1815] Using max model len 32768
INFO 10-09 11:51:44 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 10-09 11:51:49 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=3731346) INFO 10-09 11:51:50 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=3731346) INFO 10-09 11:51:50 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='/mnt/disk1/xulin/models/Qwen3-8B', speculative_config=None, tokenizer='/mnt/disk1/xulin/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, m

[W1009 11:51:52.075006004 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=3731346) INFO 10-09 11:51:52 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=3731346) WARNING 10-09 11:51:52 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=3731346) INFO 10-09 11:51:52 [gpu_model_ru

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.36it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:02,  1.29it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:02<00:01,  1.44it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:02<00:00,  1.44it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.84it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.61it/s]
(EngineCore_DP0 pid=3731346) 


(EngineCore_DP0 pid=3731346) INFO 10-09 11:51:56 [default_loader.py:268] Loading weights took 3.14 seconds
(EngineCore_DP0 pid=3731346) INFO 10-09 11:51:56 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 3.329125 seconds
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:02 [backends.py:539] Using cache directory: /home/wjyeo/.cache/vllm/torch_compile_cache/5c00c4fc42/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:02 [backends.py:550] Dynamo bytecode transform time: 5.43 s
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:04 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.165 s
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:05 [monitor.py:34] torch.compile takes 5.43 s in total
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:06 [gpu_worker.py:298] Available KV cache memory: 90.83 GiB
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:06 [kv_cache_utils.py:864] GPU KV cache size: 661,424 tokens
(Engine

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:03<00:00, 21.89it/s]


(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:10 [gpu_model_runner.py:3118] Graph capturing finished in 3 secs, took 0.77 GiB
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:10 [gpu_worker.py:391] Free memory on device (139.21/139.81 GiB) on startup. Desired GPU memory utilization is (0.8, 111.85 GiB). Actual usage is 15.27 GiB for weight, 5.68 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.77 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=96544422195` to fit into requested memory, or `--kv-cache-memory=125921838592` to fully utilize gpu memory. Current kv cache memory in use is 97532180787 bytes.
(EngineCore_DP0 pid=3731346) INFO 10-09 11:52:10 [core.py:218] init engine (profile, create kv cache, warmup model) took 13.73 seconds
INFO 10-09 11:52:10 [llm.py:295] Supported_tasks: ['generate']
INFO 10-09 11:52:10 [__init__.py:36] No IOProcessor plugins requested by the model


ERROR 10-09 14:20:34 [core_client.py:564] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.


In [4]:
ds = load_dataset('Agent-Ark/Toucan-1.5M','SFT',split = 'train').to_list()

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

In [5]:
def try_json_loads(value): # THE JSON tool call labels are messy.
    """Try to parse value as JSON, otherwise return as-is."""
    if isinstance(value, str):
        try:
            parsed = json.loads(value)
            # Recursively process the parsed result as well
            return recursively_fix(parsed)
        except (json.JSONDecodeError, TypeError):
            return value
    return value

def recursively_fix(obj):
    """Recursively decode nested JSON strings inside dicts/lists."""
    if isinstance(obj, dict):
        return {k: recursively_fix(try_json_loads(v)) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [recursively_fix(try_json_loads(x)) for x in obj]
    else:
        return try_json_loads(obj)

def fix_tool_response(tool_response):
    tool_response = deepcopy(tool_response)
    tool_response = ast.literal_eval(tool_response)
    tool_response = recursively_fix(tool_response)
    return json.dumps(tool_response, ensure_ascii=False)


In [6]:
## reformat the dataset
toolcall_format = "<tool_call>\n{tool}\n</tool_call>\n"

mode = 'dpo' # set as sft or dpo
sample_single_assistant_turns = 'qwen' in tokenizer.name_or_path.lower() and mode == 'sft' # Qwen only generate tool calls, so we can train on single assistant turns. Also, we only do this for sft since dpo, we will generate the data.

# Get the length.
def get_len(sample):
    convs = sample['conversations']
    formatted = tool_prompt_format(convs,sample['tools'],tokenizer)
    sample['len'] =len(tokenizer.encode(formatted))
    return sample

cat_ds = defaultdict(list)
invalid_cats = defaultdict(int)


for sample in tqdm(ds,total = len(ds)):
    try: # have to be able to parse both of them.
        tools = json.loads(sample['tools'])
        messages = json.loads(sample['messages'])
    except:
        invalid_cats['parsing_error'] += 1
        continue
    
    cat = sample['subset_name']
    roles = [m['role'] for m in messages]
    if not any([r == 'tool_response' for r in roles]): # no tool roles
        invalid_cats['no_tool_use'] += 1
        continue
    
    structured_msg = []
    curr_tool_call = ''
    assistant_start = False
    valid_assistant_turns = [] # We add valid assistant turns with the option of choosing any random turn to only do SFT on that turn, the issue is that in Qwen, the no-thinking mode causes extra tokens <think> and </think> to be added only to the last turn, thus we if train on all turns, usually only the tool turns are earlier and do not have these <think> tokens, thus there will be a mismatch.
    tool_started = False # we try to optimize for the case where only when there is a tool return, then we count the assistant turns. (not valid in single-turn)
    for msg_id,m in enumerate(messages): # we ignore the assistant role since Qwen do not generate any text other than the MCP tools
        if m['role'] == 'tool_call':
            tool_response = fix_tool_response(m['content'])
            curr_tool_call += toolcall_format.format(tool = tool_response)
        else:
            if curr_tool_call != '': # if the next msg is not tool_call, we dump the curr_tool_call
                structured_msg.append({'role':'assistant','content':curr_tool_call.strip()})
                curr_tool_call = ''
                if tool_started:
                    valid_assistant_turns.append(len(structured_msg)-1)
            if m['role'] == 'user':
                structured_msg.append(m)
            elif m['role'] == 'tool_response':
                tool_started = True
                structured_msg.append({'role':'tool','content':m['content']})
            elif m['role'] == 'assistant': # if is assistant, we only keep it if it is the last msg or if the next msg is not tool_call.
                if msg_id == len(messages) - 1 or messages[msg_id + 1]['role'] != 'tool_call':
                    structured_msg.append(m)
                    if tool_started:
                        valid_assistant_turns.append(len(structured_msg)-1)
    if sample_single_assistant_turns:
        if len(valid_assistant_turns) > 1:
            chosen_turn = np.random.choice(valid_assistant_turns)
            structured_msg = structured_msg[:chosen_turn+1]
            assert structured_msg[-1]['role'] == 'assistant', "Last message must be assistant!"
        if '<tool_call>' in structured_msg[-1]['content']: # split the last turn to either involving a tool call or jsut a sumamry sentence else we use the original cat
            cat_label = 'tool'
        else:
            cat_label = 'non_tool'
    else:
        cat_label = sample['subset_name']
    cat_ds[cat_label].append(
        {'conversations':structured_msg,
         'tools':tools,}
    )

for k in invalid_cats:
    print(f"Invalid cat {k}: {invalid_cats[k]} samples")

for k in cat_ds:
    print(f"Valid cat {k}: {len(cat_ds[k])} samples")

print (f"Total samples with assistant tool calls: {len(cat_ds['tool'])} / {len(ds)}")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119287/119287 [00:24<00:00, 4807.31it/s]

Invalid cat no_tool_use: 40000 samples
Valid cat single-turn-original: 28254 samples
Valid cat single-turn-diversify: 15790 samples
Valid cat multi-turn: 35243 samples
Total samples with assistant tool calls: 0 / 119287


In [9]:
# Use multi process to get the lengths
cat_ds_w_len = {k:async_process(get_len,cat_ds[k],workers=32, msg="Computing input lengths") for k in cat_ds}


Computing input lengths: 100%|███████████████████████████| 35243/35243 [00:36<00:00, 967.94it/s]


In [6]:
MAX_LENGTH = 4096
cat_ds_valid = {k:[s for s in cat_ds_w_len[k] if s['len'] <= MAX_LENGTH] for k in cat_ds_w_len}
for k in cat_ds_w_len:
    pprint (f"Cat {k}: before filtering {len(cat_ds_w_len[k])}, after filtering {len(cat_ds_valid[k])}")

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cat single-turn-original: before filtering 28254, after filtering 22863                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cat single-turn-diversify: before filtering 15790, after filtering 13314                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cat multi-turn: before filtering 35243, after filtering 21562                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# we should balance it out, since the dataset has more non-tool samples. we take an equal amount of tool and non-tool samples.
if sample_single_assistant_turns:
    min_cat_size = min([len(cat_ds_valid[k]) for k in cat_ds_valid])
    for k in cat_ds_valid:
        cat_ds_valid[k] = np.random.permutation(cat_ds_valid[k])[:min_cat_size].tolist()

concatenated_ds = sum(list(cat_ds_valid.values()),[])
random_ids = np.random.permutation(len(concatenated_ds))
val_size = 500
train_ids = random_ids[val_size:]
val_ids = random_ids[:val_size]

train_ds = [concatenated_ds[i] for i in train_ids]
val_ds = [concatenated_ds[i] for i in val_ids]

print (f"Train size: {len(train_ds)}, Val size: {len(val_ds)}")

with open('/mnt/disk1/wjyeo/data/toucan_train.json','w') as f:
    json.dump(train_ds,f,indent = 4)
with open('/mnt/disk1/wjyeo/data/toucan_val.json','w') as f:
    json.dump(val_ds,f,indent = 4)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Create adversarial samples

1. To ease the insertion of attack, we find tool_response that just returns a string as data points to insert the attack
2. We first find possible attacks : for each sample, get the user instruction, generate on-policy tool call for that. (this will be the rejected response)

In [7]:
for k in cat_ds:
    print(f"Valid cat {k}: {len(cat_ds[k])} samples")

Valid cat single-turn-original: 28254 samples
Valid cat single-turn-diversify: 15790 samples
Valid cat multi-turn: 35243 samples
Valid cat tool: 0 samples


**We need to balance between tool calling and non-tool calling chosen/rejected**

The reason is because rejected is always tool calling, if we take chosen responses that aren't tool calling, i.e. responding the user's query using the retrieved tool response, it will result in an unintended behavior where the model learns to not use tools!

And since we can only take inputs after the 1st tool response, this automatically filters out single-turn samples that do not have tool calls after the initial and only tool call. Thus for multi-tool samples, we only take after the 1st tool call and we can create multiple truncated conversations of the same sample whenver we encounter the pattern: "Editable Tools" -> assistant

In [16]:
def find_most_words(d):
    """
    Recursively find the (key_path, value, word_count) where the value
    is the longest string by word count in any nested dict/list structure.
    Supports paths like 'a.b[2].c'.
    """
    max_key_path, max_value, max_words = None, None, 0

    def recurse(obj, path=""):
        nonlocal max_key_path, max_value, max_words

        if isinstance(obj, dict):
            for k, v in obj.items():
                new_path = f"{path}.{k}" if path else k
                recurse(v, new_path)

        elif isinstance(obj, list):
            for idx, item in enumerate(obj):
                new_path = f"{path}[{idx}]" if path else f"[{idx}]"
                recurse(item, new_path)

        elif isinstance(obj, str):
            word_count = len(obj.split())
            if word_count > max_words:
                max_key_path, max_value, max_words = path, obj, word_count

    recurse(d)
    return max_key_path, max_value, max_words

import re
from copy import deepcopy

def set_by_path(d, path, new_value):
    """
    Set a nested value in a dict/list structure using a path like:
    'a.b[0].c'. Works in place and returns the modified object.
    """
    # Break path into segments:  e.g. "a.b[0].c" → ["a", "[0]", "c"]
    parts = re.findall(r'\w+|\[\d+\]', path)

    current = d
    for part in parts[:-1]:
        if part.startswith('[') and part.endswith(']'):
            idx = int(part[1:-1])
            current = current[idx]
        else:
            current = current[part]

    last = parts[-1]
    if last.startswith('[') and last.endswith(']'):
        idx = int(last[1:-1])
        current[idx] = new_value
    else:
        current[last] = new_value

    return d

    

In [44]:
def parse_tool_response(tool_resp):
    tool_resp = fix_tool_response(tool_resp)
    try:
        json_dict = json.loads(tool_resp)
        if not isinstance(json_dict, dict): # if it is not a dict, it has to be at least 5 words to be considered as text
            if len(tool_resp.split()) > 5:
                return {'type':'text', 'content':tool_resp}
            else:
                return {}
        else: # is a dict, we traverse the dict and try to find the value with the highest number of words, if it is < 5, we discard the whole thing, else we save the path so that can we can directly use that path to edit the text
            key_path, longest_text, word_count = find_most_words(json_dict)
            if word_count > 5:
                return {'type':'dict', 'content':json_dict, 'key_path':key_path,'text':longest_text}
            else:
                return {}
            
    except:
        if len(tool_resp.split()) > 5: # at least 5 splitable words, if yes, we take until the assistant turn
            return {'type':'text', 'content':tool_resp}
    return {}


def retrieve_adversarial_samples(examples): # can have multiple of the sample example if we encounter the pattern of editable tool -> assistant.
    valid_examples = []
    for sample in tqdm(examples,total = len(examples)):
        last_assistant_idx = -1
        for i,msg in enumerate(sample['conversations']):
            valid_tool_turns = []
            if i <= last_assistant_idx: # we want to skip the earlier turns that have been already added as examples
                continue
            if msg['role'] == 'tool': # once found a tool, look for the next assistant turn
                tool_resp = msg['content']
                if len(parse_tool_response(tool_resp)) > 0: # is an editable tool response
                    valid_tool_turns.append(i)
                    for j in range(i+1,len(sample['conversations'])): 
                        if sample['conversations'][j]['role'] == 'assistant':
                            valid_examples.append({ # keep the assistant turn as we want to use it for the completion attack
                                'conversations':sample['conversations'][:j+1],
                                'tools':sample['tools'],
                                'valid_tool_turns':valid_tool_turns, # keep track of which tool turns are valid for attack insertion
                            })
                            last_assistant_idx = j
                            break # if is not a valid assistant turn, we stop here and wait for the next tool call
                        elif sample['conversations'][j]['role'] == 'tool' and len(parse_tool_response(sample['conversations'][j]['content'])): # keep in mind that there might be multiple tool calls that return string, so we keep track of those valid ones
                            valid_tool_turns.append(j)


    return valid_examples

In [46]:
combined_ds = sum(list(cat_ds.values()),[])
print (f"Total combined ds size: {len(combined_ds)}")
valid_adversarial_ds = retrieve_adversarial_samples(combined_ds)
print(f"Valid adversarial samples: {len(valid_adversarial_ds)}")

def dedupe_examples(examples):
    seen = set()
    uniq = []
    for ex in examples:
        # canonicalize the conversations slice to a JSON string for hashing
        key = json.dumps(ex['conversations'], sort_keys=True, ensure_ascii=False)
        if key not in seen:
            seen.add(key)
            uniq.append(ex)
    return uniq

valid_adversarial_ds = dedupe_examples(valid_adversarial_ds)
print(f"Deduplicated Valid adversarial samples: {len(valid_adversarial_ds)}")

Total combined ds size: 79287


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79287/79287 [00:03<00:00, 22008.45it/s]


Valid adversarial samples: 102228
Deduplicated Valid adversarial samples: 75492


In [19]:
attack_samples = []
for i,sample in tqdm(enumerate(combined_ds),total = len(combined_ds)):
    for j,msg in enumerate(sample['conversations']):
        if msg['role'] == 'user':
            # next turn should be assistant and not tool response (will be weird)
            if sample['conversations'][j+1]['role'] != 'assistant':
                break
            else:
                attack_samples.append({
                    'conversations':sample['conversations'][:j+1], # dont include the assistant turn
                    'tools':sample['tools'],
                })
                break
print (len(attack_samples))   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79287/79287 [00:00<00:00, 602668.68it/s]

79287


In [47]:
# Ensure the input len is not too long
MAX_LENGTH = 4096-512
adv_ds_w_len = async_process(get_len,valid_adversarial_ds,workers=64, msg="Computing input lengths for adv ds")
attack_samples_w_len = async_process(get_len,attack_samples,workers=64, msg="Computing input lengths for attack samples")

valid_len_adversarial_ds = [s for s in adv_ds_w_len if s['len'] <= MAX_LENGTH]
print (f"Number of valid adversarial samples in cat after length filtering: {len(valid_len_adversarial_ds)} from {len(adv_ds_w_len)}")

valid_adversarial_ds = valid_len_adversarial_ds
attack_samples = [s for s in attack_samples_w_len if s['len'] <= MAX_LENGTH]
print (f"Number of valid attack samples after length filtering: {len(attack_samples)}")

Computing input lengths for attack samples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 75532/75532 [00:24<00:00, 3138.96it/s]

Number of valid adversarial samples in cat after length filtering: 45894 from 75492
Number of valid attack samples after length filtering: 75532


In [48]:
import io

OTHER_DELM_TOKENS = {
    'mark': ['{s}', '|{s}|', '<{s}>', '[{s}]', '<|{s}|>', '[|{s}|]', '<[{s}]>', '\'\'\'{s}\'\'\'', '***{s}***'],
    'inst': ['Command', 'Rule', 'Prompt', 'Task'],
    'inpt': ['Data', 'Context', 'Text'],
    'resp': ['Output', 'Answer', 'Reply'],
    'user': ['', 'Prompter ', 'User ', 'Human '],
    'asst': ['', 'Assistant ', 'Chatbot ', 'Bot ', 'GPT ', 'AI '],
}

def create_injection_for_completion(response, instruction):
    mark = np.random.choice(OTHER_DELM_TOKENS['mark']) + ':'
    
    def sample_delm(delm_name):
        role_name = 'user' if (delm_name == 'inst' or delm_name == 'inpt') else 'asst'
        role = np.random.choice(OTHER_DELM_TOKENS[role_name]) 
        delm = np.random.choice(OTHER_DELM_TOKENS[delm_name])
        p = np.random.rand()
        if p < 1/3: return (role + delm).upper()
        elif p < 2/3: return (role + delm).lower()
        else: return role + delm
    
    text = mark.format(s=sample_delm('resp')) + '\n\n' + response 
    text += '\n\n' + mark.format(s=sample_delm('inst')) + '\n\n' + instruction
    return text

def jload(f, mode="r", num_samples=None):
    if not isinstance(f, io.IOBase): f = open(f, mode=mode)
    jdict = json.load(f)
    f.close()
    return jdict

def jdump(obj, f, mode="w", indent=4, default=str):
    if not isinstance(f, io.IOBase): f = open(f, mode=mode)
    if isinstance(obj, (dict, list)): json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str): f.write(obj)
    else: raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()


def calculate_length_for_preference_dataset(dataset, tokenizer):
    chosen_input_ids = tokenizer([d['chosen'] for d in dataset], add_special_tokens=False)["input_ids"]
    rejected_input_ids = tokenizer([d['rejected'] for d in dataset], add_special_tokens=False)["input_ids"]

    chosen_lengths = np.array([len(prompt) for prompt in chosen_input_ids])
    rejected_lengths = np.array([len(prompt) for prompt in rejected_input_ids])
    prompt_and_label_lengths = np.maximum(chosen_lengths,rejected_lengths)

    print('Input+Output model_max_length (98%, 99%, 99.5%, 99.9%):', np.percentile(prompt_and_label_lengths, [95, 99, 99.5, 99.9]))
    print (f'Mean: {(np.mean(chosen_lengths) + np.mean(rejected_lengths))/2:.2f} Num > 2048: {np.sum(prompt_and_label_lengths>2048)} / {len(prompt_and_label_lengths)}')

In [49]:
"""Generate suitable attacks, suitable means that the instruction should cause the model to call a tool, else abandon those samples.
Chosen input are samples from valid_adversarial_ds and rejected inputs are from attack_samples
""" 

def find_instruction(convs):
    for msg in convs:
        if msg['role'] == 'user':
            return msg['content']
    return None

preference_input = []
for sample in tqdm(valid_adversarial_ds,total = len(valid_adversarial_ds)):
    current_sample = deepcopy(sample)
    injected_sample = np.random.choice(attack_samples,size=  1,replace=False)[0]

    clean_instruction = find_instruction(current_sample['conversations'])
    injected_instruction = find_instruction(injected_sample['conversations'])

    while injected_instruction == clean_instruction: # make sure that the injected instruction is different from the clean one
        injected_sample = np.random.choice(attack_samples,size=  1,replace=False)[0]
        injected_instruction = find_instruction(injected_sample['conversations'])
    valid_tool_turns = current_sample['valid_tool_turns']

    clean_convs = deepcopy(current_sample['conversations'])
    # turn_to_inject = valid_tool_turns[-1] # inject at the last valid tool turn
    is_text_tools = []
    for valid_t in valid_tool_turns:
        parsed_tool = parse_tool_response(clean_convs[valid_t]['content'])
        if len(parsed_tool) > 0 and parsed_tool['type'] == 'text':
            is_text_tools.append(valid_t)
    if len(is_text_tools) > 0: # prioritize text tools
        turn_to_inject = np.random.choice(is_text_tools,size=1,replace=False)[0] # randomly choose a valid tool turn to inject
    else: # else choose any valid tool turn
        turn_to_inject = np.random.choice(valid_tool_turns,size=1,replace=False)[0] # randomly choose a valid tool turn to inject
    injected_data = parse_tool_response(clean_convs[turn_to_inject]['content']) # either a string or a dict
    if injected_data['type'] == 'dict':
        injected_data_text = injected_data['text']
        injected_data_path = injected_data['key_path']
        injected_data_dict = injected_data['content']
    else:
        injected_data_text = injected_data['content']
        injected_data_path = None
    
    ## do the injection here on the injected_data_text
    if injected_data_text[-1] not in ['.','!','?']:
        injected_data_text += '.'
    if np.random.rand() < 0.9:  # 90% Straightforward Attack, 10% Completion Attack
        assert clean_convs[turn_to_inject]['role'] == 'tool', "Should be tool turn"
        injected_data_text = injected_instruction + " " + injected_data_text if np.random.rand() < 0.5 else injected_data_text + " " + injected_instruction
    else: # completion attack, use the actual output
        original_response = clean_convs[-1]['content'] # the assistant turn
        injected_data_text += '\n\n' + create_injection_for_completion(original_response, injected_instruction)
    
    ## directly replace the text or replace the value at the path
    if injected_data_path is not None: # if is a dict, we need to replace the value at the path
        try:
            injected_data_dict = set_by_path(injected_data_dict, injected_data_path, injected_data_text)
        except:
            continue # if cannot set by path, we skip this sample
        clean_convs[turn_to_inject]['content'] = json.dumps(injected_data_dict, ensure_ascii=False)
    else:
        clean_convs[turn_to_inject]['content'] = injected_data_text
        

    if clean_convs[-1]['role'] == 'assistant':
        clean_convs = clean_convs[:-1] # take away the assistant turn, as we want the model to generate it

    assert all([isinstance(t,dict) for t in current_sample['tools']]), "Tools should be dicts"
    assert all([isinstance(t,dict) for t in injected_sample['tools']]), "Tools should be dicts"
    preference_input.append(
        {'conversations':clean_convs,
        'tools':current_sample['tools'],
        'chosen_input':sample,# includes the assistant, remember to take away the assistant turn
        'chosen_tools':current_sample['tools'],
        'rejected_input':injected_sample, 
        'rejected_tools':injected_sample['tools'],
        }
    )
print (len(preference_input))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45894/45894 [04:47<00:00, 159.42it/s]

45879


In [57]:
sampling_params = SamplingParams(temperature=0.8, max_tokens=2048, stop=tokenizer.eos_token)

# First generate chosen responses (take 80% of samples where the chosen is a tool call, 20% where it is normal text)
for sample in preference_input:
    chosen_input = sample['chosen_input']['conversations'][:-1] # take away the assistant turn
    assert chosen_input[-1]['role'] != 'assistant'
    formatted_chosen = tool_prompt_format(chosen_input,sample['chosen_tools'],tokenizer)
    sample['formatted_chosen'] = formatted_chosen
    rejected_input = sample['rejected_input']['conversations']
    formatted_rejected = tool_prompt_format(rejected_input,sample['rejected_tools'],tokenizer)
    sample['formatted_rejected'] = formatted_rejected

# First generate the chosen responses
chosen_inputs = [s['formatted_chosen'] for s in preference_input]
chosen_outputs = vllm_generate(model, chosen_inputs, sampling_params,use_tqdm=True)

for sample, output in zip(preference_input, chosen_outputs):
    sample['chosen'] = output

def check_tool_use(output):
    return '<tool_call>' in output and '</tool_call>' in output

samples_with_chosen_tool_use_id = [i for i,s in enumerate(chosen_outputs) if check_tool_use(s)] # then take these samples and generate the rejected responses
print (f"Total samples with chosen tool use: {len(samples_with_chosen_tool_use_id)} / {len(chosen_outputs)}")
samples_without_chosen_tool_use_id = [i for i in range(len(chosen_outputs)) if i not in samples_with_chosen_tool_use_id]

tool_split = 0.8

tool_use_samples = [preference_input[i] for i in samples_with_chosen_tool_use_id]
non_tool_use_samples = [preference_input[i] for i in samples_without_chosen_tool_use_id]

valid_samples = tool_use_samples + non_tool_use_samples[:int(((1-tool_split)/tool_split)*len(tool_use_samples))]
print (f"Total valid samples for preference data: {len(valid_samples)}")


rejected_inputs = [s['formatted_rejected'] for s in valid_samples]
rejected_outputs = vllm_generate(model, rejected_inputs, sampling_params,use_tqdm=True)
for sample, output in zip(valid_samples, rejected_outputs):
    sample['rejected'] = output

successful_dpo_samples = []
for i in range(len(valid_samples)):
    sample = valid_samples[i]
    input_tools = deepcopy(sample['tools'])
    rejected_response = sample['rejected']
    clean_response = sample['chosen']
    ## check if rejected_response has is a tool use:
    if '<tool_call>' in rejected_response and '</tool_call>' in rejected_response: # might have multiple tools
        injected_tool_desc = []
        injected_tool_match = re.findall(r"<tool_call>\s*(.*?)\s*</tool_call>", rejected_response, re.DOTALL)
        if len(injected_tool_match):
            for injected_tool in injected_tool_match:
                try:
                    injected_tool = json.loads(injected_tool)
                    injected_tool_name = injected_tool.get('name', None)
                    if injected_tool_name is not None:
                        for tool in sample['rejected_tools']:
                            if tool['function']['name'] == injected_tool_name:
                                assert isinstance(tool,dict)
                                injected_tool_desc.append(tool)
                except Exception as e:
                    pass
            if len(injected_tool_desc) > 0:
                input_tools.extend(injected_tool_desc)
                input_tools = [input_tools[j] for j in np.random.permutation(len(input_tools))] # shuffle the tools
    assert isinstance(input_tools,list) and len(input_tools) > 0 and all([isinstance(t,dict) for t in input_tools]), "Input tools should be non-empty list of dicts"
    # if not tool use, no need to add the additional tool
    successful_dpo_samples.append(
        {
            'prompt':sample['conversations'],
            'tools': input_tools,
            'chosen':clean_response,
            'rejected':rejected_response,
        }
    )

val_size = 500 # redo the val size
preference_data_ids = np.random.permutation(range(len(successful_dpo_samples)))
train_dataset = [successful_dpo_samples[i] for i in range(len(successful_dpo_samples)) if i not in preference_data_ids[:val_size]]
val_dataset = [successful_dpo_samples[i] for i in preference_data_ids[:val_size]]

preference_data_path = {
    'train':'/mnt/disk1/wjyeo/data/toucan_dpo_train.json',
    'val':'/mnt/disk1/wjyeo/data/toucan_val_train.json', # change this later
}

jdump(train_dataset, preference_data_path['train'])
jdump(val_dataset, preference_data_path['val'])
train_dataset = jload(preference_data_path['train'])
val_dataset = jload(preference_data_path['val'])
calculate_length_for_preference_dataset(train_dataset, tokenizer)
calculate_length_for_preference_dataset(val_dataset, tokenizer)

Total valid samples for preference data: 15888


Adding requests:   0%|          | 0/15888 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Input+Output model_max_length (98%, 99%, 99.5%, 99.9%): [511. 512. 512. 512.]
Mean: 94.51 Num > 2048: 0 / 15388
Input+Output model_max_length (98%, 99%, 99.5%, 99.9%): [512. 512. 512. 512.]
Mean: 96.22 Num > 2048: 0 / 500
